In [135]:
# Import the libraries
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV

from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


## Import data

In [136]:
#13k top spotify songs 
#df=pd.read_csv('data/df.csv', sep=",")
#users favorite tracks based on time 
df_fav=pd.read_csv('data/df_fav.csv', sep=",")
df=pd.read_csv('data/df_hot.csv', sep=",")

In [137]:
df.describe()

,length,artists,album,time_signature,danceability,acousticness,energy,tempo,instrumentalness,loudness,...,valence,speechiness,mode,Popularity,minutes,beat_density,Popularity_Rank,melody_complexity,instrumental_acoustic_ratio,favorite
count,12594.00,12594.00,12594.00,12594.00,12594.00,12594.00,12594.00,12594.00,12594.00,12594.00,...,12594.00,12594.00,12594.00,12594.00,12594.00,12594.00,12594.00,12594.00,12594.00,12594.00
mean,238555.06,4135.18,4870.25,4.50,0.55,0.32,0.59,119.01,0.18,-9.63,...,0.47,0.08,0.67,50.30,3.98,35.10,6180.61,465.29,537.97,0.01
std,126349.68,2385.11,2834.52,6.38,0.19,0.36,0.28,31.16,0.32,6.95,...,0.43,0.11,0.68,19.92,2.11,18.80,3633.37,132.17,11978.91,0.10
min,9750.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-44.15,...,0.00,0.00,0.00,0.00,0.16,0.00,1.00,0.00,0.00,0.00
25%,177104.00,2094.25,2406.00,4.00,0.43,0.02,0.41,97.48,0.00,-10.96,...,0.22,0.04,0.00,39.00,2.95,24.42,3045.50,380.04,0.00,0.00
50%,211946.50,4201.50,4908.50,4.00,0.57,0.13,0.66,121.94,0.00,-7.21,...,0.42,0.05,1.00,52.00,3.53,33.46,6092.00,480.02,0.01,0.00
75%,259297.00,6213.75,7320.75,4.00,0.69,0.61,0.83,137.69,0.18,-5.20,...,0.64,0.08,1.00,65.00,4.32,42.87,9370.50,543.82,0.92,0.00
max,4170227.00,8136.00,9764.00,96.00,0.98,1.00,1.00,219.45,0.99,1.18,...,5.00,1.00,11.00,100.00,69.50,574.66,12343.00,926.36,747863.25,1.00


In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12594 entries, 0 to 12593
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   track_id                     12594 non-null  object 
 1   length                       12594 non-null  int64  
 2   artists                      12594 non-null  int64  
 3   album                        12594 non-null  int64  
 4   time_signature               12594 non-null  int64  
 5   danceability                 12594 non-null  float64
 6   acousticness                 12594 non-null  float64
 7   energy                       12594 non-null  float64
 8   tempo                        12594 non-null  float64
 9   instrumentalness             12594 non-null  float64
 10  loudness                     12594 non-null  float64
 11  liveness                     12594 non-null  float64
 12  key                          12594 non-null  float64
 13  valence         

In [139]:
df['favorite'].value_counts()

favorite
0    12476
1      118
Name: count, dtype: int64

In [140]:
print(df.shape)

(12594, 23)


In [141]:
df

,track_id,length,artists,album,time_signature,danceability,acousticness,energy,tempo,instrumentalness,...,valence,speechiness,mode,Popularity,minutes,beat_density,Popularity_Rank,melody_complexity,instrumental_acoustic_ratio,favorite
0,3rUGC1vUpkDG9CZFHMur1t,131872,7016,9646,1,0.75,0.26,0.73,111.02,0.00,...,0.84,0.03,0,98.00,2.20,50.51,4.50,111.02,0.00,0
1,7aqfrAY2p9BUSiupwk3svU,247444,2075,2891,4,0.47,0.03,0.64,163.92,0.00,...,0.25,0.32,1,91.00,4.12,39.75,49.50,655.68,0.00,0
2,7dJYggqjKo71KI9sLzqCs8,265360,2007,7025,4,0.75,0.23,0.67,123.03,0.00,...,0.39,0.10,0,89.00,4.42,27.82,89.50,492.10,0.00,0
3,4MjDJD8cW7iVeWInc2Bdyj,267194,742,9691,4,0.79,0.15,0.62,139.06,0.00,...,0.13,0.07,0,94.00,4.45,31.23,21.00,556.22,0.00,0
4,5aIVCx5tnk0ntmdiinnYvw,200256,7579,9135,4,0.67,0.09,0.72,117.19,0.00,...,0.52,0.08,0,92.00,3.34,35.11,37.50,468.75,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12589,5R2rsbwCDXORX2tLfprRmM,166343,15,5774,72,0.72,0.32,0.51,0.00,0.15,...,4.00,1.00,7,0.46,2.77,0.00,39.00,0.00,0.48,1
12590,6DZz58CbF0AKw5PCKqNKcM,310393,12,2325,0,0.44,0.69,0.47,0.00,0.38,...,4.00,1.00,3,0.24,5.17,0.00,69.00,0.00,0.56,1
12591,3CA9pLiwRIGtUBiMjbZmRw,210746,6,7135,82,0.58,0.09,0.91,0.00,0.12,...,4.00,1.00,8,0.76,3.51,0.00,10.00,0.01,1.34,1
12592,27GmP9AWRs744SzKcpJsTZ,205879,6,9212,74,0.85,0.06,0.55,0.00,0.33,...,4.00,1.00,1,0.66,3.43,0.00,21.00,0.00,5.94,1


In [142]:
#X = df.drop(columns=['favorite', 'track_id'])  # Drop 'favorite' and 'track_id' columns
#X = df[['valence', 'key', 'time_signature','artists','tempo','beat_density','speechiness','melody_complexity']]
X = df[['album', 'artists', 'length', 'danceability', 'tempo', 'energy', 'acousticness', 'time_signature']]
#'tempo','beat_density','speechiness','melody_complexity'

y = df['favorite']  # Select the 'favorite' column as the target variable

In [143]:
print(X.shape)
print(y.shape)

(12594, 8)
(12594,)


Train & Testing data set 

In [144]:
# Shuffle your dataset 
shuffle_df = df.sample(frac=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(y_train.shape)
print(y_test.shape)

(10075,)
(2519,)


In [145]:
#make sure the columns are assigned correctly 
X_train = pd.DataFrame(X_train, columns=['album', 'artists', 'length', 'danceability', 'tempo', 'energy', 'acousticness', 'time_signature'])
X_test = pd.DataFrame(X_test, columns=['album', 'artists', 'length', 'danceability', 'tempo', 'energy', 'acousticness', 'time_signature'])
#

In [146]:
X_train.head(3)

,album,artists,length,danceability,tempo,energy,acousticness,time_signature
0,-0.93,-1.43,0.13,-0.56,-1.34,0.82,-0.85,-0.08
1,0.55,-1.48,-0.01,0.14,0.19,1.22,-0.87,-0.08
2,-0.17,1.62,-0.47,1.19,0.67,-0.25,0.67,-0.08


##### fix the imbalanced data set 

In [147]:
from imblearn.over_sampling import SMOTE
# Train / Split Data
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X, y) 

In [148]:
y_train.value_counts()

favorite
0    12476
1    12476
Name: count, dtype: int64

In [149]:
y_test.value_counts()

favorite
0    2499
1      20
Name: count, dtype: int64

In [150]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(24952, 8)
(24952,)
(2519, 8)
(2519,)


## ML Algo

In [151]:
import lazypredict
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,predictions=True, ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

 14%|█▍        | 4/29 [00:00<00:05,  4.25it/s]

100%|██████████| 29/29 [01:35<00:00,  3.28s/it]

[LightGBM] [Info] Number of positive: 12476, number of negative: 12476
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000868 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1874
[LightGBM] [Info] Number of data points in the train set: 24952, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

## KNN's  

In [152]:
# Assuming '_pca' is already fitted on your data

# Get the feature names
feature_names = df.columns  # Assuming 'df' is your DataFrame

# Get the indices of the top features in each principal component
top_feature_indices = [np.abs(_pca.components_[i]).argsort()[::-1][:8] for i in range(components)]

# Get the names of the top features in each principal component
selected_features = [[feature_names[j] for j in top_feature_indices[i]] for i in range(components)]

# Print the selected features for each component
for i, features in enumerate(selected_features):
    print(f"Top features for component {i}: {features}")


Top features for component 0: ['acousticness', 'danceability', 'album', 'artists', 'time_signature', 'track_id', 'length', 'energy']
Top features for component 1: ['energy', 'time_signature', 'length', 'album', 'artists', 'acousticness', 'track_id', 'danceability']
Top features for component 2: ['track_id', 'length', 'artists', 'time_signature', 'energy', 'album', 'danceability', 'acousticness']
Top features for component 3: ['length', 'track_id', 'time_signature', 'energy', 'album', 'artists', 'acousticness', 'danceability']
Top features for component 4: ['artists', 'acousticness', 'danceability', 'energy', 'track_id', 'album', 'length', 'time_signature']
Top features for component 5: ['album', 'energy', 'time_signature', 'artists', 'danceability', 'length', 'acousticness', 'track_id']
Top features for component 6: ['time_signature', 'album', 'energy', 'danceability', 'acousticness', 'artists', 'length', 'track_id']
Top features for component 7: ['acousticness', 'danceability', 'album

In [154]:
# Initialize the KNN model and fit it to the training data
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn.fit(X_train, y_train)

# Predict the target variable for the test set
y_pred = knn.predict(X_test)


In [155]:
# Evaluate the model (you can use classification metrics to assess its performance)
# For instance, you can use accuracy_score, classification_report, confusion_matrix, etc.
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Further evaluation
print(classification_report(y_test, y_pred, zero_division=1)) 
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9920603414053196
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2499
           1       1.00      0.00      0.00        20

    accuracy                           0.99      2519
   macro avg       1.00      0.50      0.50      2519
weighted avg       0.99      0.99      0.99      2519

[[2499    0]
 [  20    0]]


In [156]:
from sklearn.model_selection import cross_val_score, KFold

# Assuming X and y are your features and target variable

# Initialize the KNN model
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors

# Define the number of folds for cross-validation
num_folds = 5

# Initialize KFold
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(knn, X, y, cv=kf)

# Output the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())

Cross-Validation Scores: [0.99206034 0.99166336 0.99086939 0.98809051 0.99046863]
Mean Accuracy: 0.9906304460997136


In [157]:
%%time
# Logistic Regression
lr = LogisticRegression(solver='lbfgs', max_iter=400).fit(X_train, y_train)
lr_scores = cross_val_score(lr, X_train, y_train, cv=10, scoring="f1")
print(np.mean(lr_scores))

NameError: name 'LogisticRegression' is not defined

## Descision tree 

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, classification_report


# Create and fit the Decision Tree classifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
# Make predictions
predictions = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [ ]:
report = classification_report(y_test, predictions)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3738
           1       1.00      1.00      1.00        41

    accuracy                           1.00      3779
   macro avg       1.00      1.00      1.00      3779
weighted avg       1.00      1.00      1.00      3779



In [165]:
X = df.drop(columns=['favorite', 'track_id'])  # Drop 'favorite' and 'track_id' columns
#X = df[['valence', 'key', 'time_signature','artists','tempo','beat_density','speechiness','melody_complexity']]
#X = df[['album', 'artists', 'length', 'danceability', 'tempo', 'energy', 'acousticness', 'time_signature']]
#'tempo','beat_density','speechiness','melody_complexity'

y = df['favorite']

### PipeLine

In [166]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

components = 8  # Define the number of components for PCA

_sc = StandardScaler()
_pca = PCA(n_components=components)
_model = KNeighborsClassifier(n_neighbors=5)

Pipe = Pipeline([
    ('std_scaler', _sc),
    ('pca', _pca),
    ('regressor', _model)
])

In [167]:
# perform a split
random_state = 12

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, 
                     test_size=0.3,
                     shuffle=True, 
                     random_state=random_state)
    
# train the model using the PCA components
Pipe.fit(X_train,y_train)


Pipeline(steps=[('std_scaler', StandardScaler()), ('pca', PCA(n_components=8)),
                ('regressor', KNeighborsClassifier())])

In [168]:
Pipe.score(X_test,y_test)

1.0

### Predict Songs 

In [169]:
df.columns

Index(['track_id', 'length', 'artists', 'album', 'time_signature',
       'danceability', 'acousticness', 'energy', 'tempo', 'instrumentalness',
       'loudness', 'liveness', 'key', 'valence', 'speechiness', 'mode',
       'Popularity', 'minutes', 'beat_density', 'Popularity_Rank',
       'melody_complexity', 'instrumental_acoustic_ratio', 'favorite'],
      dtype='object')

In [170]:
# Predicting if a song is a favorite
prob_preds = Pipe.predict_proba(df.drop(['favorite','track_id'], axis=1))
threshold = 0.30 # define threshold here
preds = [1 if prob_preds[i][1]> threshold else 0 for i in range(len(prob_preds))]
df['prediction'] = preds

In [171]:
df['prediction']

0        0
1        0
2        0
3        0
4        0
        ..
12589    1
12590    1
12591    1
12592    1
12593    1
Name: prediction, Length: 12594, dtype: int64

In [172]:
df['prediction'].value_counts()

prediction
0    12476
1      118
Name: count, dtype: int64

#### Building the playlist from recommended songs

In [173]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2

In [174]:
# Declare the credentials
client_id = '2a7adde1b07a4bf9a5fd750d6ff6f28d'
client_secret = '74a5c5859a0548ccae791419cc29c8fa'
redirect_uri='http://localhost:7777/callback'
username = 'ethan.falcao'

In [175]:
# Once the Authorisation is complete, we just need to `sp` to call the APIs
scope = 'user-top-read playlist-modify-private playlist-modify-public'
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [176]:
# Creating a function that builds a playlist in the user's spotify account
def create_playlist(sp, username, playlist_name, playlist_description):
    playlists = sp.user_playlist_create(username, playlist_name, description = playlist_description)

In [177]:
create_playlist(sp, username, 'Your New Jams', 'This playlist was created using python!')

In [178]:
# Function to check if the playlist was created successfully
def fetch_playlists(sp, username):
    """
    Returns the user's playlists.
    """
        
    id = []
    name = []
    num_tracks = []
    
    # Make the API request
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:
        id.append(playlist['id'])
        name.append(playlist['name'])
        num_tracks.append(playlist['tracks']['total'])

    # Create the final df   
    df_playlists = pd.DataFrame({"id":id, "name": name, "#tracks": num_tracks})
    return df_playlists

In [179]:
fetch_playlists(sp,username).head()

,id,name,#tracks
0,7q9MublCR8vYmsEfZI3QkS,Your New Jams,0
1,1AJFPBjF116BRPJsbjoOn7,July 2023,20
2,1oxbP11MEsbkUqyMlaIYac,October 2023,27
3,06gBlEcmbpmpIxgDuMnqrw,September 2023,32
4,30VWwWahI4rVSsEjPO9WIP,August 2023,11


In [180]:
# Getting the playlist ID of the most recently made playlist so we can add songs to it later
playlist_id = fetch_playlists(sp,username)['id'][0]

In [181]:
# Function to add selected songs to playlist
def enrich_playlist(sp, username, playlist_id, playlist_tracks):
    index = 0
    results = []
    
    while index < len(playlist_tracks):
        results += sp.user_playlist_add_tracks(username, playlist_id, tracks = playlist_tracks[index:index + 50])
        index += 50

In [182]:
# Adding songs to playlist
list_track = df.loc[df['prediction']  == 1]['track_id']
enrich_playlist(sp, username, playlist_id, list_track)